## Object Relation Model

In [2]:
import os
from sqlmodel import create_engine, Field, Session, select, SQLModel, Relationship

### Single entity

In [2]:
class Customer(SQLModel, table=True):
    """Class for managing customer data."""

    Id: int | None = Field(primary_key=True)
    Name: str
    Lastname: str
    Age: int | None

In [3]:
# Create a (SQLAlchemy) engine
engine = create_engine("sqlite:///orm-test.db", echo=True)

In [ ]:
# Create tables based on the model metadata.
SQLModel.metadata.create_all(engine)

In [ ]:
# Create a customer object
customer = Customer(Name="Ingrid", Lastname="Jansen")
customer

In [ ]:
# Create a session to the database.
with Session(engine) as session:
    # Add the customer record and commit.
    session.add(customer)
    session.commit()

In [ ]:
sql_customers = []
with Session(engine) as session:
    # Create the select query.
    query = select(Customer)

    # Perform the query on the database.
    result = session.exec(query)

    # Fetch all results.
    sql_customers = result.fetchall()


In [ ]:
# Note that the results are Python / SQLModel objects!
sql_customers

In [9]:
# Delete the engine / close connection
engine.dispose(close=True)
os.remove("orm-test.db")

### Foreign Keys

In [ ]:
class Address(SQLModel, table=True):
    """Class for managing address data."""

    __tablename__ = "Addresses"

    AddressId: int | None = Field(primary_key=True)
    Street: str
    HouseNumber: str
    PostCode: str


class Customer(SQLModel, table=True):
    """Class for managing customer data."""

    __tablename__ = "Customers"

    CustomerId: int | None = Field(primary_key=True)
    Name: str
    Lastname: str
    Age: int | None

    # Create a foreign key constraint.
    AddressId: int = Field(foreign_key="Addresses.AddressId")

In [ ]:
engine = create_engine("sqlite:///orm-test.db", echo=True)
SQLModel.metadata.create_all(engine)

In [4]:
# Create a session.
session = Session(engine)

In [5]:
# Define the address.
address = Address(Street="Teststreet", HouseNumber=42, PostCode="1234 AB")

# Define the customer without the address ID.
customer = Customer(Name="John", Lastname="Doe", Age=42)

In [ ]:
# Add the address first to generate an ID.
session.add(address)

# Flush the session to get the generated ID.
session.flush()

print("Generated Address ID: ", address.AddressId)

In [7]:
# Link the address to the customer.
customer.AddressId = address.AddressId

# Now store the customer.
session.add(customer)

In [ ]:
# Commit the changes.
session.commit()

In [ ]:
# Read linked tables using where().
query =  select(Customer, Address).where(Customer.AddressId == Address.AddressId)
results = session.exec(query)

for result in results.fetchall():
    customer, address = result
    print("-" * 80)
    print ("Customer: ", customer)
    print ("Address:  ", address)


In [ ]:
# Read linked tables using join().
query =  select(Customer, Address).join(Address)
results = session.exec(query)

for result in results.fetchall():
    customer, address = result
    print("-" * 80)
    print ("Customer: ", customer)
    print ("Address:  ", address)

In [ ]:
# Delete the engine / close connection
session.close()
engine.dispose(close=True)
os.remove("orm-test.db")

### Relationships

In [ ]:
class Address(SQLModel, table=True):
    """Class for managing address data."""

    __tablename__ = "Addresses"

    AddressId: int | None = Field(primary_key=True)
    Street: str
    HouseNumber: str
    PostCode: str

    # Define Customer relationship as an object property.
    Customer: "Customer" = Relationship(back_populates="Adress")


class Customer(SQLModel, table=True):
    """Class for managing customer data."""

    __tablename__ = "Customers"

    CustomerId: int | None = Field(primary_key=True)
    Name: str
    Lastname: str
    Age: int | None

    # Create a foreign key constraint.
    AddressId: int = Field(foreign_key="Addresses.AddressId")

    # Define Address relationship as an object property.
    Adress: Address = Relationship(back_populates="Customer")

In [ ]:
engine = create_engine("sqlite:///orm-test.db", echo=True)
SQLModel.metadata.create_all(engine)

In [5]:
# Create session
session = Session(engine)

In [6]:
# Define the address.
address = Address(Street="Teststreet", HouseNumber=42, PostCode="1234 AB")

# Define the customer.
# Note: The address object is passed along using the relationship!
customer = Customer(Name="John", Lastname="Doe", Age=42, Adress=address)

In [ ]:
# Store the customer
# Note: The address is stored automatically!
session.add(customer)
session.commit()

In [ ]:
query = select(Customer)
customer = session.exec(query).one()
customer

In [ ]:
# Get the address via the relationship
customer.Adress

In [ ]:
# Access address properties directly.
customer.Adress.Street

In [ ]:
# Delete the engine / close connection
session.close()
engine.dispose(close=True)
os.remove("orm-test.db")